# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235217533355                   -0.50    7.0
  2   -7.250379500540       -1.82       -1.40    1.0
  3   -7.251226379162       -3.07       -2.03    4.0
  4   -7.251038544812   +   -3.73       -1.93    5.0
  5   -7.251334517026       -3.53       -2.81    5.0
  6   -7.251338240025       -5.43       -3.18    4.0
  7   -7.251338796920       -6.25       -4.24    4.0
  8   -7.251338797820       -9.05       -4.60    3.0
  9   -7.251338798698       -9.06       -5.74    3.0
 10   -7.251338798704      -11.26       -6.04    5.0
 11   -7.251338798704      -12.19       -6.47    3.0
 12   -7.251338798705      -13.11       -6.82    2.0
 13   -7.251338798705      -13.69       -7.62    3.0
 14   -7.251338798705   +  -15.05       -8.06    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04873025322556542
[ Info: Arnoldi iteration step 2: normres = 0.49696288488849477
[ Info: Arnoldi iteration step 3: normres = 0.9612629029025941
[ Info: Arnoldi iteration step 4: normres = 0.3092852018505094
[ Info: Arnoldi iteration step 5: normres = 0.3135940603482085
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.02e-02, 3.67e-02, 2.41e-01, 1.94e-01, 2.28e-02)
[ Info: Arnoldi iteration step 6: normres = 0.45570923452081397
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.05e-02, 4.49e-02, 4.29e-01, 1.16e-01, 8.42e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08873403721264422
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.05e-04, 7.65e-03, 3.55e-02, 3.72e-02, 6.08e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09488620760518059
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.10e